In [1]:
# | default_exp agent.planner.validation

In [2]:
# | export
import re
from typing import List
from rich.console import Console

from agentic.tools.introspect import IntrospectTool
from agentic.tools.base import ToolMetadata, ToolCategory

from agentic.agent.planner.models import Task, IntrospectionResult, ProjectBreakdown, ProjectContext


In [3]:
# | export 
class TaskValidator:
    """Handles task validation and consistency checks"""
    
    def __init__(self, console: Console):
        self.console = console
    
    def validate_task_consistency(self, task: Task) -> IntrospectionResult:
        """Validate internal consistency of task fields"""
        issues = []
        
        # Only check if task description matches action purposes
        if not any(keyword in task.description.lower() for keyword in ['create', 'generate', 'build', 'implement', 'setup']):
            if any('write' in action.purpose.lower() or 'create' in action.purpose.lower() for action in task.actions):
                issues.append("Task uses creation actions but description doesn't indicate creation/setup")
        
        success = len(issues) == 0
        feedback = f"Task consistency issues: {'; '.join(issues)}" if issues else "Task fields are consistent"
        
        return IntrospectionResult(
            success=success,
            score=10.0 if success else 5.0,
            feedback=feedback,
            next_action="regenerate" if not success else "proceed",
            recommendations=["Ensure file names match across all task fields", "Include specific files in expected_outputs if mentioned in success_criteria"] if issues else []
        )
    
    def introspect_task_planning(self, task: Task, breakdown: ProjectBreakdown, context: ProjectContext) -> IntrospectionResult:
        """Pre-execution validation of task planning"""
        
        consistency_result = self.validate_task_consistency(task)
        if not consistency_result.success:
            return consistency_result
        
        # Simple validation without LLM introspection
        if len(task.description) < 10:
            return IntrospectionResult(
                success=False,
                score=3.0,
                feedback="Task description too short - needs more detail",
                next_action="regenerate",
                recommendations=["Add more detailed task description"]
            )
        
        if not task.expected_outputs:
            return IntrospectionResult(
                success=False,
                score=4.0,
                feedback="No expected outputs defined",
                next_action="regenerate", 
                recommendations=["Define expected outputs for the task"]
            )
        
        # Task passes basic validation
        return IntrospectionResult(
            success=True,
            score=8.0,
            feedback="Task validation passed",
            next_action="proceed",
            recommendations=[]
        )

